<a href="https://colab.research.google.com/github/crispitagorico/Neural-SPDEs/blob/main/examples/example_hyperparameter_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1>Stochastic Korteweg–De Vries equation</h1> </center>

*The stochastic Korteweg-De Vries (KdV) equations are used to describe the propagation of nonlinear waves at the surface of a fluid subject to random perturbations.*

*In this notebook we fit a Neural SPDE model to learn dynamics described by the KdV equations,*
\begin{align*}
    \partial_t u + \gamma\partial_x^3 u &= 6u\partial_xu + \xi,  \\
    u(t,0) &= u(t,1),  \nonumber\\
    u(0,x) &= u_0(x), \quad (t,x)\in [0,T]\times[0,1]\,. \nonumber
\end{align*}
*and compare the performances against other models. For each model we perform a grid search over its hyperparameters using a validation set. All models are run on a GPU.*
***

In [2]:
# clone the github repository if using Google Colab

In [1]:
%cd Neural-SPDEs/

/content/Neural-SPDEs


In [ ]:
!pip install -r requirements.txt

In [3]:
import torch
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [4]:
import torch.optim as optim
from utilities import *
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

**Load the KdV dataset**

In [5]:
data = scipy.io.loadmat('../drive/MyDrive/data_kdv/kdv_xi_1200.mat')

In [6]:
O_X, O_T, W, Sol = data['X'], data['T'], data['W'], data['sol']

In [7]:
xi = torch.from_numpy(W.astype(np.float32))
data = torch.from_numpy(Sol.astype(np.float32))

# 1) Neural SPDE
***

In [8]:
from torchspde.neural_spde import *

In [9]:
print('Number of train, val and test instances: {}, {}, {}'.format(0.70*1200, 0.15*1200, 0.15*1200))

Number of train, val and test instances: 840.0, 180.0, 180.0


In [ ]:
dir = '../drive/MyDrive/data_kdv/' # directory where results are saved

T, sub_t = 50, 1  # number of time steps to predict

filename = 'nspde_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

_, test_dl = dataloader_nspde_1d(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                 ntest=test_nb, T=T, sub_t=sub_t, 
                                 batch_size=20, dim_x=128)

train_dl, val_dl = dataloader_nspde_1d(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], 
                                       ntrain=train_nb, ntest=val_nb, T=T, sub_t=sub_t, 
                                       batch_size=20, dim_x=128)

hyperparameter_search_nspde(train_dl, val_dl, test_dl, 
                            d_h=[32], iter=[1,2,3,4], modes1=[32, 64], modes2=[32,50],
                            epochs=500, print_every=20, plateau_patience=50, 
                            plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                            checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                            final_checkpoint_file=dir+'best_'+filename+'.pt')

# 2) FNO
***

In [11]:
from baselines.FNO1D import *

In [ ]:
dir = '../drive/MyDrive/data_kdv/' # directory where results are saved

T, sub_t = 50, 1  # number of time steps to predict

filename = 'fno_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

_, test_dl = dataloader_fno_1d_xi(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                  ntest=test_nb, T=T, sub_t=sub_t, 
                                  batch_size=20, dim_x=128)

train_dl, val_dl = dataloader_fno_1d_xi(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], 
                                        ntrain=train_nb, ntest=val_nb, T=T, sub_t=sub_t, 
                                        batch_size=20, dim_x=128)

hyperparameter_search_fno1d(train_dl, val_dl, test_dl, T=50, 
                            d_h=[32], iter=[1,2,3,4], modes1=[16, 32], modes2=[16,25],  
                            lr=0.0025, epochs=500, print_every=20, plateau_patience=50, 
                            plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                            checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                            final_checkpoint_file=dir+'best_'+filename+'.pt')

# 3) NCDE
***

In [13]:
from baselines.NCDE import *

In [ ]:
dir = '../drive/MyDrive/data_kdv/' # directory where results are saved

T, sub_t = 50, 1  # number of time steps to predict

filename = 'ncde_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

dim_x = 128

_, test_dl, norm = dataloader_ncde_1d(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                      ntest=test_nb, T=T, sub_t=sub_t, normalizer=True,
                                      batch_size=20, dim_x=dim_x, interpolation='linear')

train_dl, val_dl, norm = dataloader_ncde_1d(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], 
                                            ntrain=train_nb, ntest=val_nb, T=T, sub_t=sub_t, normalizer=True,
                                            batch_size=20, dim_x=dim_x, interpolation='linear')

hyperparameter_search_ncde(train_dl, val_dl, test_dl, dim_x, norm,
                           d_h=[8,16,32], solver=['euler', 'rk4'], lr=0.0025,
                           epochs=1000, print_every=20, plateau_patience=50, 
                           plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                           checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                           final_checkpoint_file=dir+'best_'+filename+'.pt')

# 4) NRDE
***

In [ ]:
# if using the NRDE model, one should install signatory
# !pip install signatory  # version 1.2.6.1.9.0 was installed

In [ ]:
from baselines.NRDE import *

In [ ]:
import signatory

**Get the dataloaders**

In [ ]:
dir = '../drive/MyDrive/data_kdv/'

T, sub_t = 50, 1  # number of time steps to predict

filename = 'nrde_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

dim_x = 128

_, test_dl, I, noise_dim, norm = dataloader_nrde_1d(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                                    ntest=test_nb, T=T, sub_t=sub_t, normalizer=None,
                                                    depth=2, window_length=3, 
                                                    batch_size=20, dim_x=dim_x, interpolation='linear')

train_dl, val_dl, I, noise_dim, norm = dataloader_nrde_1d(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], 
                                                          depth=2, window_length=5, 
                                                          ntrain=train_nb, ntest=val_nb, 
                                                          T=T, sub_t=sub_t, normalizer=None,
                                                          batch_size=20, dim_x=dim_x, interpolation='linear')


hyperparameter_search_nrde(train_dl, val_dl, test_dl, noise_dim, I, dim_x, norm,
                           d_h=[1], lr=0.001,
                           epochs=1000, print_every=20, plateau_patience=50, 
                           plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                           checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                           final_checkpoint_file=dir+'best_'+filename+'.pt')

# 5) NCDE-FNO
***

In [15]:
from baselines.NCDEFNO_1D import *

In [ ]:
dir = '../drive/MyDrive/data_kdv/'

T, sub_t = 50, 1  # number of time steps to predict

filename = 'ncdefno_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

_, test_dl = dataloader_ncdeinf_1d(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                   ntest=test_nb, T=T, sub_t=sub_t, 
                                   batch_size=20, dim_x=128, interpolation='linear')

train_dl, val_dl = dataloader_ncdeinf_1d(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], 
                                         ntrain=train_nb, ntest=val_nb, T=T, sub_t=sub_t, 
                                         batch_size=20, dim_x=128, interpolation='linear')

hyperparameter_search_ncdefno_1d(train_dl, val_dl, test_dl,
                                 d_h=[8,16,32], solver=['euler', 'rk4'], lr=0.0025, 
                                 epochs=500, print_every=20, plateau_patience=50, 
                                 plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                                 checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                                 final_checkpoint_file=dir+'best_'+filename+'.pt')

# 6) DeepONet
***

In [17]:
from baselines.deepOnet import *

In [ ]:
dir = '../drive/MyDrive/data_kdv/'

T, sub_t = 50, 1  # number of time steps to predict

filename = 'deeponet_example_kdv_{}_{}'.format(str(T),str(sub_t))

train_nb, val_nb, test_nb = 840, 180, 180

_, test_dl, norm, grid = dataloader_deeponet_1d_xi(u=data, xi=xi, ntrain=train_nb+val_nb, 
                                                   ntest=test_nb, T=T, sub_t=sub_t, 
                                                   batch_size=20, dim_x=128, normalizer=True)

train_dl, val_dl, norm, grid = dataloader_deeponet_1d_xi(u=data[:train_nb+val_nb], xi=xi[:train_nb+val_nb], ntrain=train_nb, ntest=val_nb, T=T, sub_t=sub_t, 
                                                         batch_size=20, dim_x=128, normalizer=True)

hyperparameter_search_deeponet(train_dl, val_dl, test_dl, dim_x*(T-1), grid, norm, 
                               width=[128,256,512], branch_depth=[2,3,4], trunk_depth=[2,3,4],
                               lr=0.0025, epochs=500, print_every=20, plateau_patience=50, 
                               plateau_terminate=100, log_file=dir+'log_'+filename+'.csv',
                               checkpoint_file=dir+'checkpoint_'+filename+'.pt', 
                               final_checkpoint_file=dir+'best_'+filename+'.pt')